In [1]:
!pip install --quiet vietocr

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
einops-exts 0.0.4 requires einops>=0.4, but you have einops 0.2.0 which is incompatible.
pyannote-audio 3.3.1 requires einops>=0.6.0, but you have einops 0.2.0 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import os
import cv2
import numpy as np

def crop_bounding_boxes(image_path, boxes, output_folder):
    # Load the image
    image = cv2.imread(image_path)

    # Create the output folder
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Crop the bounding boxes
    for i, box in enumerate(boxes):
        # Convert box coordinates to integer
        box = np.array(box).astype(int)
        
        # Calculate width and height of the bounding box
        width = int(np.linalg.norm(box[0] - box[1]))
        height = int(np.linalg.norm(box[0] - box[3]))
        
        # Ensure the text is always horizontal
        src_pts = box.astype("float32")
        dst_pts = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype="float32")
        M = cv2.getPerspectiveTransform(src_pts, dst_pts)
        
        # Apply the perspective transformation
        cropped_image = cv2.warpPerspective(image, M, (width, height))
        
        if height * 1.0 / width >= 1.5:
            cropped_image = cv2.rotate(cropped_image, cv2.ROTATE_90_CLOCKWISE)            
        
        # Save the cropped image
        cv2.imwrite(f'{output_folder}/{i}.png', cropped_image)


In [2]:
def create_all_folders_if_not_exists(path):
    os.makedirs(path, exist_ok=True)

In [3]:
def find_all_text_detection(root_dir, extensions=[".json"]):
    text_detection_files = []
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(filename.lower().endswith(ext) for ext in extensions):
                text_detection_files.append(os.path.join(dirpath, filename))
    return text_detection_files

In [4]:
text_detections = find_all_text_detection("/media/daoan/T7 Shield2/AI_Challenge_2024_DATA/Text_detection")

In [5]:
import torch
# Import necessary libraries
from vietocr.tool.predictor import Predictor
from vietocr.tool.config import Cfg
from PIL import Image

# Load the pre-trained VietOCR model
config = Cfg.load_config_from_name('vgg_seq2seq')
config['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'
detector = Predictor(config)

10935it [01:04, 169.46it/s]


In [6]:
import json

def recognize_text(image_path):
    # Load the image
    image = Image.open(image_path)

    # Perform text recognition
    text = detector.predict(image)

    return text

In [7]:
import json
from tqdm import tqdm

for text_detection in tqdm(text_detections, desc="Processing text_recognition"):
    text_reg = text_detection.replace(".json", ".txt").replace("Text_detection", "Text_reg")
    if os.path.exists(text_reg):
        continue
    create_all_folders_if_not_exists(os.path.dirname(text_reg))
    keyframe = text_detection.replace(".json", ".jpg").replace("Text_detection", "Keyframes")
    
    if not os.path.exists(keyframe):
        continue
    with open(text_detection, "r") as f:
        data = json.load(f)
        # check tmp folder exists
        if os.path.exists("tmp"):
            os.system(f"rm -rf tmp")
        crop_bounding_boxes(keyframe, data, "tmp")
        total_text = ""
        for i, file in enumerate(os.listdir("tmp")):
            text = recognize_text(f"tmp/{file}")
            total_text += text + " "
        total_text = total_text.strip()
        if len(total_text) > 0:
            with open(text_reg, "w", encoding="utf-8") as f:
                f.write(total_text)
     

Processing text_recognition:  28%|██▊       | 33681/121269 [34:07<1:23:25, 17.50it/s] /home/daoan/Projects/AI_Challenge_HCMC_2024/.venv/lib/python3.10/site-packages/vietocr/tool/translate.py:115: RuntimeWarning: invalid value encountered in divide
  char_probs = np.sum(char_probs, axis=-1)/(char_probs>0).sum(-1)
Processing text_recognition: 100%|██████████| 121269/121269 [1:59:53<00:00, 16.86it/s]  
